This is an example on how to use the CAFE60 dataset. It is very long to read the netcdf files directly on AWS but there is a way to process the netcdf files to pretend they are zarr files. zarr files are a lot faster to read on cloud storage.

I've put together this package `cafaz` to allow you to read data that has already been processed and to process new variables to fake zarr.

In [1]:
import xarray as xr
import climtas.nci
import dask
import cafaz

Always open a Dask client as there are a lot of files to open, this ensures they will be read in parallel.

In [2]:
client = climtas.nci.Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/ood-vn10/29618/proxy/8787/status,
Dashboard: /node/ood-vn10/29618/proxy/8787/status,Workers: 8
Total threads: 8,Total memory: 22.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39961,Workers: 8
Dashboard: /node/ood-vn10/29618/proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 22.46 GiB
Comm: tcp://127.0.0.1:36145,Total threads: 1
Dashboard: /node/ood-vn10/29618/proxy/35691/status,Memory: 2.81 GiB
Nanny: tcp://127.0.0.1:34305,


# Read the data

For a data that was already processed, you first define an object for the variable you want and then you read it. It will give you a whole bunch of warnings but nothing to worry about.

There is an additional `json_root` optional argument to the Cafaz class. It defaults to where I stored the json files for the processed variables but you will want to specify it if you process additional variables (see the processing new variables section below).

The `read_CAFE()` method accept a period argument to read only a subset of the data. It needs to be given as a `slice`, e.g `slice("1980-02","1985-08")`

In [3]:
precip_test = cafaz.Cafaz("precip","atmos_hybrid","month")
ds1 = precip_test.read_CAFE()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/dask/utils.py:37: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  return func(*args, **kwargs)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/dask/utils.py:37: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata i

In [4]:
ds1

<xarray.Dataset>
Dimensions:      (latb: 133, lonb: 145, lat: 124, nv: 2, ensemble: 96, lon: 144, time: 726)
Coordinates:
  * latb         (latb) float64 -90.0 -88.99 -86.97 -84.94 ... 86.97 88.99 90.0
  * lonb         (lonb) float64 0.0 2.5 5.0 7.5 10.0 ... 352.5 355.0 357.5 360.0
  * lat          (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * nv           (nv) float64 1.0 2.0
  * ensemble     (ensemble) float64 1.0 2.0 3.0 4.0 5.0 ... 93.0 94.0 95.0 96.0
  * lon          (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * time         (time) object 1960-01-16 12:00:00 ... 2020-11-16 00:00:00
Data variables:
    precip       (time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 96, 117, 132), meta=np.ndarray>
    time_bounds  (time, nv) timedelta64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/27)
    NCO:                       4.3.2
    calendar:                   julian
    cm-enkf_source:            commit 0141830a243704acbf6d8c8c843b161a39ec39fa
    contact_name:              Decadal Activity 1 - Data Assimilation
    control_name:              c5
    creation_date:             Wed Mar 13 00:00:18 2019
    ...                        ...
    nco_openmp_thread_number:  1
    nominal_resoltuion:        Atmosphere delta lat = 2.02degrees ; Atmospher...
    perturbation_name:         not applicable
    references:                OKane, T.J., Sandery, P.A., Monselesan, D.P., ...
    run_variant_name:          data assimilation
    title:                     AccessOcean-AM2

Then you can use the data as usual. Be careful though that the data still resides on AWS so it will take longer to process analyses because of the reading time.
# Process new variables

To process new variables, you simply need 2 lines of codes. Again make sure you start a Dask client previously. The `json_root` option is to define where to write the json files so choose a path you can write to. The code will create the directory if it does not exist.

In [5]:
sst = cafaz.Cafaz("sst","ocean","month",json_root="/g/data/w35/ccc561/CAFE60/json")
sst.write_json()

Files from 1960 to 1967 (inclusive) don't have `geolat_t` which prevent us to read the whole timeseries. One needs to use the `period` argument in that case when reading in the data.

In [6]:
sst.read_CAFE(period=slice("1968","2000"))

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'geolat_t' has multiple fill values {9.96921e+36, 1.0000000200408773e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'geolon_t' has multiple fill values {9.96921e+36, 1.0000000200408773e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'sst' has multiple fill values {9.96921e+36, -1.0000000200408773e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'geolat_t' has mult

<xarray.Dataset>
Dimensions:      (nv: 2, ensemble: 96, yt_ocean: 300, xt_ocean: 360, time: 396)
Coordinates:
  * nv           (nv) float64 1.0 2.0
  * ensemble     (ensemble) float64 1.0 2.0 3.0 4.0 5.0 ... 93.0 94.0 95.0 96.0
    geolat_t     (yt_ocean, xt_ocean) float32 dask.array<chunksize=(95, 114), meta=np.ndarray>
    geolon_t     (yt_ocean, xt_ocean) float32 dask.array<chunksize=(95, 114), meta=np.ndarray>
  * time         (time) object 1968-01-16 12:00:00 ... 2000-12-16 12:00:00
  * xt_ocean     (xt_ocean) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * yt_ocean     (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
Data variables:
    sst          (time, ensemble, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 96, 95, 114), meta=np.ndarray>
    time_bounds  (time, nv) timedelta64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/27)
    NCO:                        netCDF Operators version 4.7.8 (Homepage = ht...
    calendar:                    julian
    cm-enkf_source:             commit 0141830a243704acbf6d8c8c843b161a39ec39fa
    contact_name:               Decadal Activity 1 - Data Assimilation
    control_name:               c5
    creation_date:              Fri Apr 19 16:48:59 2019
    ...                         ...
    model_source:               commit 2abb29f4384e68777721fa657850587f28efd85a
    nominal_resoltuion:         Atmosphere delta lat = 2.02degrees ; Atmosphe...
    perturbation_name:          not applicable
    references:                 OKane, T.J., Sandery, P.A., Monselesan, D.P.,...
    run_variant_name:           data assimilation
    title:                      AccessOcean-AM2